The Georgia Birds and the Michigan Felines play a game where they flip a fair coin 101 times. In the end, if heads comes up at least 51 times, the Birds win; but if tails comes up at least 51 times, the Felines win.

What’s the probability that the Birds have at least a 99 percent chance of winning at some point during the game — meaning their probability of victory is 99 percent or greater given the flips that remain — and then proceed to lose?

Extra credit: Instead of 101 total flips, suppose there are many, many more (i.e., consider the limit as the number of flips goes to infinity). Again, the Birds win if heads comes up at least half the time. Now what’s the probability that the Birds have a win probability of at least 99 percent at some point and then proceed to lose?

# Main problem

In [1]:
from scipy.linalg import pascal
from scipy.special import comb

In [2]:
def win_prob(heads_required, flips_left):
    prob = sum(pascal(flips_left + 1, kind='lower')[-1][heads_required:]) / (2 ** flips_left)
    return prob

In [3]:
TURNS = 101
n_heads_to_win = TURNS // 2 + 1
THRESHOLD = 0.99
turn_combs = {}
heavy_favourite_states = []

# one way to throw a coin zero times
turn_combs[0] = [1]

# calculate ways to throw a coin N times without going above THRESHOLD probability of winning
for turn in range(1, TURNS+1):
    last_turn = turn_combs[turn-1]
    max_heads_last_turn = len(last_turn) - 1
    turn_combs[turn] = []
    for n_heads in range(len(last_turn)+1):
        if n_heads > max_heads_last_turn and win_prob(n_heads_to_win - n_heads, TURNS - turn) > THRESHOLD:
            if n_heads_to_win > n_heads:
                heavy_favourite_states.append((last_turn[-1], 2 ** turn, turn))
            break
        turn_combs[turn].append(sum(last_turn[max(n_heads-1,0):n_heads+1]))

In [4]:
possible_games = 2 ** TURNS
ways_to_lose = possible_games / 2
ways_to_lose_without_blowing_a_lead = sum(turn_combs[TURNS])
ways_to_blow_a_lead = ways_to_lose - ways_to_lose_without_blowing_a_lead
prob_of_blowing_a_lead = ways_to_blow_a_lead / possible_games
prob_of_blowing_a_lead

0.002117155741949872

# Extra

```
p(lose) = 0.5 = p(lose|heavy_lead).p(heavy_lead) + p(lose|~heavy_lead).p(~heavy_lead)

As n -> infinity:
 - p(lose|heavy_lead) -> (1 - threshold)  # i.e. 1 - 0.99 = 0.01 in the example given
 - p(lose|~heavy_lead) -> 1  # share of paths that never have a heavy lead -> 0
 
let h = p(heavy_lead)
let t = the threshold for a heavy lead

as n -> infinity, 0.5 = (1 - t) * h + (1 - h)
                      = 1 - th
                      = 1 - 0.99h

Therefore h -> 0.5/t = 0.5/0.99

p(blowing a lead) -> h(1-t) = (1-t)/2t
                            = 0.01/1.98
                            = 0.005050505050505051
```

In [5]:
0.01/1.98

0.005050505050505051